In [1]:
_ALPHABET = {
    "A": ["a"],
    "B": ["be", "bé"],
    "C": ["ce", "se", "sé", "cé", "si", "ci"],
    "D": ["de", "dé", "di"],
    "E": ["e", "é"],
    "F": ["eff", "ef"],
    "G": ["ge", "gé", "gi"],
    "H": ["hå", "ho"],
    "I": ["i"],
    "J": ["ji", "gi"],
    "K": ["kå", "ko"],
    "L": ["ell", "el"],
    "M": ["emm", "em"],
    "N": ["enn", "en"],
    "O": ["o"],
    "P": ["pe", "pé", "pi"],
    "Q": ["qu"],
    "R": ["err", "er", "är", "ärr"],
    "S": ["ess", "es"],
    "T": ["te", "té", "ti"],
    "U": ["u"],
    "V": ["ve", "vé", "vi"],
    "W": ["dubbelve"],
    "X": ["ex", "ecz", "ecs", "eks"],
    "Y": ["y"],
    "Z": ["zäta", "säta", "seta", "zeta"],
    "Å": ["å"],
    "Ä": ["ä"],
    "Ö": ["ö"]
}

DIGITS = {
    "1": ["ett"],
    "2": ["två"],
    "3": ["tre"],
    "4": ["fyra"],
    "5": ["fem"],
    "6": ["sex"],
    "7": ["sju"],
    "8": ["åtta"],
    "9": ["nio", "ni"]
}

In [2]:
ALPHABET = {k: sorted(v, key=len, reverse=True) for k,v in _ALPHABET.items()}

```
2442206120015761721 1 487.78 0.32 essefu 1.0 <eps> ins
2442206120015761721 1 488.26 0.16 fem 1.0 SfU5 sub
```

```
2442206120015761721 1 490.6 0.519 tjugohundrafjorton 1.0 <eps> ins
2442206120015761721 1 491.2 0.379 femton 1.0 <eps> ins
2442206120015761721 1 491.74 0.779 etthundratjugofyra 1.0 2014/15:124 sub
```

In [3]:
ALNUM = {**ALPHABET, **DIGITS}
ALNUM_REGEX = {k: f"({'|'.join(v)})" for k,v in ALNUM.items()}

In [4]:
from difflib import SequenceMatcher

ACCEPT = [
    ("e", "ə")
]

a = "kamaren"
b = "kamarən"

ok = False

s = SequenceMatcher(None, a, b)
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag == "replace":
        if (a[i1:i2], b[j1:j2]) in ACCEPT:
            ok = True
        else:
            ok = False
    elif tag == "equal":
        ok = True

In [5]:
from phonemizer import phonemize

In [6]:
phonemize("slut", language='sv')

'slʉt '

In [7]:
from sync_asr.utils.nst_lexicon import get_nst_lexicon

In [9]:
lexicon = get_nst_lexicon()

In [39]:
dictionary = {}
for entry in lexicon:
    if 'garbage' in entry and entry['garbage'] == 'GARB':
        continue
    else:
        word = entry['orthography']
        word = word.replace("_", " ")
        if not word in dictionary:
            dictionary[word] = set()
        for translit in entry['transliterations']:
            dictionary[word].add(translit['ipa'].replace(".", "").replace("¤", "").replace("_", " ").replace("\u0361", ""))

In [33]:
CONSONANTS = "bdfhjklmnprstvŋɕɖɡɧɭɳʂʈ"
VOWELS = "aeiouyøɪɑɔɛɵʉʊʏ"
ACCENTS = "²ˌˈ"

In [106]:
print(dictionary["inte"])
print(phonemize("inte", language='sv'))

{'²ɪnte'}
ɪntɛ 


In [29]:
characters = set()
for entry in dictionary:
    for pron in dictionary[entry]:
        for char in pron:
            characters.add(char)

In [34]:
cons = set()
VS = VOWELS + ACCENTS + "_"
for entry in dictionary:
    for pron in dictionary[entry]:
        for char in pron:
            if char not in VS:
                cons.add(char)

In [41]:
def shift_accent(ipa):
    accent = ""
    output = ""
    for character in ipa:
        if character in ACCENTS:
            accent = character
        elif character in CONSONANTS:
            output += character
        elif character in VOWELS:
            if accent != "":
                output += accent
                accent = ""
            output += character
        else:
            output += character
    return output

In [42]:
shift_accent("²kratʊʂ")

'kr²atʊʂ'